In [1]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import threading

# 初始化Flask应用
app = Flask(__name__, template_folder='./templates')  # 指定模板文件夹

# 加载数据
df = pd.read_csv("douban_top250_processed.csv", encoding="utf-8-sig")
movie_features = pd.read_csv("movie_features.csv", encoding="utf-8-sig")

# 预计算相似度矩阵
def build_similarity_matrix():
    numeric_features = movie_features[["rating", "year", "rating_people"]].values
    numeric_features = (numeric_features - numeric_features.min(axis=0)) / (numeric_features.max(axis=0) - numeric_features.min(axis=0) + 1e-8)
    onehot_cols = [col for col in movie_features.columns if col.startswith("genre_") or col.startswith("region_")]
    onehot_features = movie_features[onehot_cols].values
    all_features = np.hstack([numeric_features, onehot_features])
    return cosine_similarity(all_features)

similarity_matrix = build_similarity_matrix()

# 路由：首页
@app.route("/")
def index():
    df["hot_score"] = df["rating"] * np.log1p(df["rating_people"])
    hot_movies = df.sort_values("hot_score", ascending=False).head(10)[["title", "rating", "year", "region"]].to_dict("records")
    return render_template("index.html", hot_movies=hot_movies)

# 路由：推荐功能
@app.route("/recommend", methods=["POST"])
def recommend():
    data = request.get_json()
    movie_title = data.get("title", "")
    try:
        movie_idx = df[df["title"].str.contains(movie_title, na=False)].index[0]
        similarity_scores = similarity_matrix[movie_idx]
        similar_indices = similarity_scores.argsort()[::-1][1:6]
        recommendations = df.iloc[similar_indices][["title", "rating", "year", "genre", "quote"]].to_dict("records")
        return jsonify({"status": "success", "data": recommendations})
    except IndexError:
        return jsonify({"status": "error", "message": f"未找到电影《{movie_title}》"})

# 路由：评分分布
@app.route("/rating_dist")
def rating_dist():
    rating_bins = [8.0, 8.5, 9.0, 9.5, 10.0]
    rating_counts = pd.cut(df["rating"], bins=rating_bins, include_lowest=True).value_counts().sort_index()
    labels = [str(interval) for interval in rating_counts.index]
    values = rating_counts.values.tolist()
    return jsonify({"labels": labels, "values": values})

# 启动Flask（用线程避免阻塞Notebook）
def run_flask():
    app.run(debug=False, host="0.0.0.0", port=5000)

# 启动Flask服务
thread = threading.Thread(target=run_flask)
thread.start()

print("Flask服务已启动！请访问 http://localhost:5000")

Flask服务已启动！请访问 http://localhost:5000
 * Serving Flask app '__main__'
 * Debug mode: off


In [2]:
html_content = '''<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>豆瓣电影推荐系统</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/echarts@5.4.3/dist/echarts.min.js"></script>
    <style>
        body { padding-top: 20px; }
        .container { max-width: 1200px; }
        #ratingChart { height: 400px; margin: 20px 0; }
        .movie-card { margin-bottom: 20px; }
    </style>
</head>
<body>
    <div class="container">
        <nav class="navbar navbar-expand-lg navbar-dark bg-dark mb-4">
            <div class="container-fluid">
                <a class="navbar-brand" href="/">🎬 电影推荐系统</a>
            </div>
        </nav>

        <div class="row mb-5">
            <div class="col-md-8 offset-md-2">
                <h3>相似电影推荐</h3>
                <div class="input-group mb-3">
                    <input type="text" id="movieTitle" class="form-control" placeholder="输入电影名称（如：肖申克的救赎）">
                    <button class="btn btn-primary" onclick="getRecommend()">搜索</button>
                </div>
                <div id="recommendResult"></div>
            </div>
        </div>

        <div class="row mb-5">
            <div class="col-12">
                <h3>电影评分分布</h3>
                <div id="ratingChart"></div>
            </div>
        </div>

        <div class="row">
            <div class="col-12">
                <h3>热门电影Top10</h3>
                <div class="row" id="hotMovies">
                    {% for movie in hot_movies %}
                    <div class="col-md-3">
                        <div class="card movie-card">
                            <div class="card-body">
                                <h5 class="card-title">{{ movie.title }}</h5>
                                <p class="card-text">评分：{{ movie.rating }} | 年份：{{ movie.year }}</p>
                                <p class="card-text"><small class="text-muted">地区：{{ movie.region }}</small></p>
                            </div>
                        </div>
                    </div>
                    {% endfor %}
                </div>
            </div>
        </div>
    </div>

    <script>
        var ratingChart = echarts.init(document.getElementById('ratingChart'));
        fetch('/rating_dist')
            .then(response => response.json())
            .then(data => {
                ratingChart.setOption({
                    title: { text: '豆瓣Top250评分分布' },
                    xAxis: { type: 'category', data: data.labels },
                    yAxis: { type: 'value' },
                    series: [{
                        name: '电影数量',
                        type: 'bar',
                        data: data.values,
                        itemStyle: { color: '#4CAF50' }
                    }]
                });
            });

        function getRecommend() {
            var title = document.getElementById('movieTitle').value.trim();
            if (!title) {
                alert('请输入电影名称！');
                return;
            }
            fetch('/recommend', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ title: title })
            })
            .then(response => response.json())
            .then(data => {
                var resultDiv = document.getElementById('recommendResult');
                if (data.status === 'success') {
                    var html = '<div class="row">';
                    data.data.forEach(movie => {
                        html += `
                        <div class="col-md-3">
                            <div class="card movie-card">
                                <div class="card-body">
                                    <h5 class="card-title">${movie.title}</h5>
                                    <p class="card-text">评分：${movie.rating} | 年份：${movie.year}</p>
                                    <p class="card-text">类型：${movie.genre}</p>
                                    <p class="card-text"><small class="text-muted">${movie.quote || '无简介'}</small></p>
                                </div>
                            </div>
                        </div>`;
                    });
                    html += '</div>';
                    resultDiv.innerHTML = html;
                } else {
                    resultDiv.innerHTML = `<div class="alert alert-danger">${data.message}</div>`;
                }
            });
        }
    </script>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>'''

# 将内容写入index.html
with open("templates/index.html", "w", encoding="utf-8") as f:
    f.write(html_content)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.27.102.241:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2025 14:46:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:46:59] "GET /rating_dist HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:46:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Dec/2025 14:47:25] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:47:49] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:47:55] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:47:59] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:48:03] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 14:48:15] "POST /recommend HTTP/1.1" 200 -
